In [15]:
import sys
import os
import json
import overpass
import geojson
import pandas as pd

from pprint import pprint
from langchain.llms import Ollama
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from geopy.geocoders import Nominatim


overp = overpass.API()
llm = Ollama(model="llama2") ## try mistral or openchat and see the differences :) 

In [16]:
def call_overpass(api,lat,long,radius=500):
    result = api.get(f"node(around:{radius},{lat},{long})")
    res = pd.json_normalize(result,['features'])
    result_string = ''
    if 'properties.name' in res.columns:
        pd_around = res[res['properties.name'].notnull()].fillna("")
        for index, item in pd_around.iterrows():
            result_string += f"Name: {item['properties.name']} Shop: {item['properties.shop']} Amenity: {item['properties.cuisine']} {item['properties.amenity']}"+"\n"
    return result_string

In [19]:
def create_chain(llm,document):
    # I save the document to a file because I cannot figure out a good way to load it into LangChain
    # as a string/text :( 
    file_path = 'output.txt'
    with open(file_path, 'w') as file:
        file.write(document)
    loader = TextLoader("./output.txt")
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    all_splits = text_splitter.split_documents(data)
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(), chain_type="stuff",
        return_source_documents=True)
    return qa_chain

In [18]:
def main():
    latitude = float(input("Enter Latitude: "))
    longitude = float(input("Enter Longitude: "))
    my_doc = call_overpass(overp,latitude,longitude)
    qa_chain = create_chain(llm,my_doc)
    geolocator = Nominatim(user_agent="http")
    location = geolocator.reverse([latitude,longitude])
    response = qa_chain(f"""All names contained in the document are businesses around {location}, can you
        please describe all available businesses and shops in the surroundings in plain English? Use a descriptional tone.""")
    print(response['result'])

main()

Enter Latitude:  37.7879
Enter Longitude:  -122.4076


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


/Users/mirko/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Sure, I'd be happy to help! Based on the context provided, here are some of the businesses and shops located around Union Square Garage, Post Street, Union Square, San Francisco, California, 94108, United States:

* Amenity: Shop
	+ Location: Stockton Street & Geary Street / Union Square
	+ Description: A shop offering various amenities for customers.
* Amenity: Shop
	+ Location: 4th Street & Market Street / Moscone Center
	+ Description: A shop located in the heart of Union Square, offering a variety of goods and services.
* Amenity: Shop
	+ Location: Kearny Street & Geary Street
	+ Description: A shop providing various amenities for customers.
* Amenity: Shop
	+ Location: 3rd Street & Mission Street
	+ Description: A shop offering a range of goods and services.
* Amenity: Shop
	+ Location: Kearny Street & Bush Street
	+ Description: A shop providing various amenities for customers.
* Amenity: Shop
	+ Location: Kearny Street & Sutter Street
	+ Description: A shop offering a range of g

In [ ]:
## Union Square, SF: 37.787937, -122.407677